In [ ]:
# Imports

import os
import sys
import numpy as np
import yaml
import copy
from tqdm import tqdm

sys.path.append("../openWakeWord")
import openwakeword
from openwakeword.data import mmap_batch_generator
from openwakeword.train import Model

import matplotlib.pyplot as plt
import torch
from torch.utils.flop_counter import FlopCounterMode
torch.cuda.device_count() # use GPU for much faster training

# Load Config for the Model

In [ ]:
# Load the YAML config file

with open('../wakeword_training_configs/angela.yml', 'r') as file:
    config = yaml.safe_load(file)

# Define paths for saving/loading files
feature_save_dir = os.path.join(config["output_dir"], config["model_name"])


# Create Model

In [ ]:
# Create the model

F = openwakeword.utils.AudioFeatures(device='cpu')
input_shape = np.load(os.path.join(feature_save_dir, "positive_features_val.npy")).shape
n_time_steps = input_shape[1]

oww = Model(n_classes=1, input_shape=(input_shape[1], input_shape[2]), model_type=config["model_type"],
            layer_dim=config["layer_size"], seconds_per_example=1280*input_shape[0]/16000)


In [ ]:
# Calculate the MACs for the model

def get_macs(model, inp, with_backward=False):
    # Compute the MACs for the model
    istrain = model.training
    model.eval()
    
    inp = inp if isinstance(inp, torch.Tensor) else torch.randn(inp)

    flop_counter = FlopCounterMode(model)
    with flop_counter:
        if with_backward:
            model(inp).sum().backward()
        else:
            model(inp)
    total_flops =  flop_counter.get_total_flops()
    if istrain:
        model.train()
    return total_flops/2  # Using the assumption that FLOPS = 2*MACs

model_macs = get_macs(oww.model, (1, input_shape[1], input_shape[2]))
print(f"\n\nTotal Model MACs: {model_macs}")

# Creat Data Transformation and Loading Functions

In [ ]:
# Create data transform function for batch generation to handle differ clip lengths
from functools import partial

def f(x, n=n_time_steps, step_size=None):
    """Simple transformation function to ensure data is the appropriate shape for the model size"""
    if step_size is None:
        step_size = n

    if n > x.shape[1] or n < x.shape[1]:
        x = np.vstack(x)
        new_batch = np.array([x[i:i+n, :] for i in range(0, x.shape[0]-n, step_size)])
    else:
        return x
    return new_batch

# Create label transforms as needed for model (currently only supports binary classification models)
data_transforms = {key: f for key in config["feature_data_files"].keys()}
label_transforms = {}
for key in ["positive"] + list(config["feature_data_files"].keys()) + ["adversarial_negative"]:
    if key == "positive" or key == "xttsv2_cv_clones":
        label_transforms[key] = lambda x: [1 for i in x]
    else:
        label_transforms[key] = lambda x: [0 for i in x]

# Add generated positive and adversarial negative clips to the feature data files dictionary
config["feature_data_files"]['positive'] = os.path.join(feature_save_dir, "positive_features_train.npy")
config["feature_data_files"]['adversarial_negative'] = os.path.join(feature_save_dir, "negative_features_train.npy")

# Make PyTorch data loaders for training and validation data
batch_generator = mmap_batch_generator(
    config["feature_data_files"],
    n_per_class=config["batch_n_per_class"],
    data_transform_funcs=data_transforms,
    label_transform_funcs=label_transforms
)

class IterDataset(torch.utils.data.IterableDataset):
    def __init__(oww, generator):
        oww.generator = generator

    def __iter__(oww):
        return oww.generator

X_train = torch.utils.data.DataLoader(IterDataset(batch_generator),
                                        batch_size=None, num_workers=1, prefetch_factor=16)  # warning, increasing num_workers will duplicate data!

X_val_fp = np.load(config["false_positive_validation_data_path"])
X_val_fp = np.array([X_val_fp[i:i+n_time_steps] for i in range(0, X_val_fp.shape[0]-n_time_steps, 1)])  # reshape to match model
X_val_fp_labels = np.zeros(X_val_fp.shape[0]).astype(np.float32)
X_val_fp = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.from_numpy(X_val_fp), torch.from_numpy(X_val_fp_labels)),
    batch_size=len(X_val_fp_labels)
)

X_val_pos = np.load(os.path.join(feature_save_dir, "positive_features_val.npy"))
X_val_neg = np.load(os.path.join(feature_save_dir, "negative_features_val.npy"))
labels = np.hstack((np.ones(X_val_pos.shape[0]), np.zeros(X_val_neg.shape[0]))).astype(np.float32)

X_val = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.from_numpy(np.vstack((X_val_pos, X_val_neg))),
        torch.from_numpy(labels)
        ),
    batch_size=len(labels)
)


In [ ]:
# Train the model with autotrain, which will automatically adjust the learning rate, negative weight, and other hyperparameters

best_model = oww.auto_train(
    X_train=X_train,
    X_val=X_val,
    false_positive_val_data=X_val_fp,
    steps=config["steps"],
    max_negative_weight=config["max_negative_weight"],
    target_fp_per_hour=config["target_false_positives_per_hour"],
)


In [ ]:
# # Alternatively, train the model without the autotrain function

# # Set the false positive validation data duration
# val_set_hrs = 11.3

# # Train with cosine decay learning rate and linear increase in negative weight
# lr = 0.0001
# weights = np.linspace(1, config["max_negative_weight"], int(config["steps"])).tolist()
# val_steps = np.linspace(config["steps"]-int(config["steps"]*0.25), config["steps"], 20).astype(np.int64)
# oww.train_model(
#             X=X_train,
#             X_val=X_val,
#             false_positive_val_data=X_val_fp,
#             max_steps=config["steps"],
#             negative_weight_schedule=weights,
#             val_steps=val_steps, warmup_steps=config["steps"]//5,
#             hold_steps=config["steps"]//3, lr=lr, val_set_hrs=val_set_hrs)

# # Merge best models from training

# accuracy_percentile = np.percentile(oww.history["val_accuracy"], 90)
# recall_percentile = np.percentile(oww.history["val_recall"], 90)
# fp_percentile = np.percentile(oww.history["val_fp_per_hr"], 10)

# # Get models above the 90th percentile
# models = []
# for model, score in zip(oww.best_models, oww.best_model_scores):
#     if score["val_accuracy"] >= accuracy_percentile and \
#             score["val_recall"] >= recall_percentile and \
#             score["val_fp_per_hr"] <= fp_percentile:
#         models.append(model)

# if len(models) > 0:
#     best_model = oww.average_models(models=models)
# else:
#     best_model = oww.model

In [ ]:
# Optionally plot some evaluation metrics

plt.plot([float(i) for i in oww.history['val_n_fp']])
# plt.plot([float(i) for i in oww.history['val_recall']])
# plt.plot([float(i) for i in oww.history['val_fp_per_hr']])
# plt.plot([float(i) for i in oww.history['loss']])


In [ ]:
# Export the trained model to onnx
oww.export_model(model=best_model, model_name=config["model_name"], output_dir="../trained_models")